In [0]:
# 03_Gold_Aggregation (Secure Version)
from pyspark.sql.functions import col, to_date, lit, concat, sum, count, min, max
from delta.tables import *

# --- 1. CONFIGURATION (SECURE) ---
try:
    client_id = dbutils.secrets.get(scope="kv-secrets", key="sp-client-id")
    client_secret = dbutils.secrets.get(scope="kv-secrets", key="sp-client-secret")
    print("✅ Secrets retrieved successfully from Key Vault.")
except Exception as e:
    print("❌ Error retrieving secrets. Check your Scope Name and Key Names.")
    raise e

# Database Config
jdbcHostname = "server-crypto-trades.database.windows.net"
jdbcDatabase = "db-crypto-trades"
jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:1433;database={jdbcDatabase};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"

connectionProperties = {
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver",
  "authentication" : "ActiveDirectoryServicePrincipal",
  "userName" : client_id,
  "password" : client_secret
}

# --- 2. HELPER: Pre-Clean Tables ---
# We use spark._jvm to access Java classes from Python
def execute_sql_cleanup():
    print("🧹 Cleaning SQL Tables (Child -> Parent)...")
    try:
        # 1. Get the Driver Manager from JVM
        driver_manager = spark._jvm.java.sql.DriverManager
        
        # 2. Setup Properties for Authentication
        props = spark._jvm.java.util.Properties()
        props.put("user", client_id)
        props.put("password", client_secret)
        props.put("authentication", "ActiveDirectoryServicePrincipal")
        
        # 3. Connect and Execute
        conn = driver_manager.getConnection(jdbcUrl, props)
        stmt = conn.createStatement()
        
        # Order matters! Delete Fact (Child) first.
        stmt.executeUpdate("DELETE FROM dbo.Fact_Daily_Market")
        stmt.executeUpdate("DELETE FROM dbo.Dim_Exchange")
        stmt.executeUpdate("DELETE FROM dbo.Dim_CurrencyPair")
        
        conn.close()
        print("✅ Tables Cleared.")
    except Exception as e:
        print(f"⚠️ Warning during cleanup: {e}")

# Run the cleanup immediately
execute_sql_cleanup()

# --- 3. Read Silver Data ---
df_silver = spark.read.table("crypto_cat.silver.trades")

# --- 4. LOAD DIMENSION: Exchange ---
df_exchanges = df_silver.select("exchange").distinct().withColumnRenamed("exchange", "ExchangeName")

print("Loading Dim_Exchange...")
(df_exchanges.write.jdbc(url=jdbcUrl, table="dbo.Dim_Exchange", mode="append", properties=connectionProperties))

# --- 5. LOAD DIMENSION: CurrencyPair ---
df_pairs = (df_silver
    .select("ccy", "quoteCcy")
    .distinct()
    .withColumn("PairName", concat(col("ccy"), lit("-"), col("quoteCcy")))
    .withColumnRenamed("ccy", "BaseCurrency")
    .withColumnRenamed("quoteCcy", "QuoteCurrency")
)

print("Loading Dim_CurrencyPair...")
(df_pairs.write.jdbc(url=jdbcUrl, table="dbo.Dim_CurrencyPair", mode="append", properties=connectionProperties))

# --- 6. PREPARE FACT TABLE ---
df_agg = (df_silver
    .withColumn("TradeDate", to_date(col("timestamp")))
    .withColumn("PairName", concat(col("ccy"), lit("-"), col("quoteCcy")))
    .groupBy("exchange", "PairName", "TradeDate")
    .agg(
        sum("qty").alias("TotalVolume"),
        sum("quoteQty").alias("TotalValueUSD"),
        count("exchange").alias("TradeCount")
    )
    .withColumn("VWAP", col("TotalValueUSD") / col("TotalVolume"))
)

# --- 7. JOIN WITH DIMS (To get IDs) ---
# Read back the Dims we just populated to get the SQL-generated IDs
dim_exchange_sql = spark.read.jdbc(url=jdbcUrl, table="dbo.Dim_Exchange", properties=connectionProperties)
dim_pair_sql = spark.read.jdbc(url=jdbcUrl, table="dbo.Dim_CurrencyPair", properties=connectionProperties)

df_fact = (df_agg.alias("f")
    .join(dim_exchange_sql.alias("e"), col("f.exchange") == col("e.ExchangeName"))
    .join(dim_pair_sql.alias("p"), col("f.PairName") == col("p.PairName"))
    .select(
        col("f.TradeDate"),
        col("e.ExchangeID"),
        col("p.PairID"),
        col("f.TotalVolume"),
        col("f.TotalValueUSD"),
        col("f.VWAP"),
        col("f.TradeCount")
    )
)

# --- 8. LOAD FACT TABLE ---
print("Loading Fact_Daily_Market...")
(df_fact.write.jdbc(url=jdbcUrl, table="dbo.Fact_Daily_Market", mode="append", properties=connectionProperties))

print("✅ Star Schema Loaded Successfully!")

# COUNT & EXIT (Send Value to ADF)
final_count = df_fact.count()
print(f"✅ Job Complete. Rows Loaded: {final_count}")

dbutils.notebook.exit(str(final_count))